In [40]:
import requests
from selenium import webdriver
import pandas as pd
import matplotlib.pyplot as plt
import re
import pickle
import nltk
from nltk.corpus import stopwords
import string
from collections import Counter
from itertools import chain

In [34]:
from ipynb.fs.full.functions import next_page, clean_date, clean_titulo, sort_df, find_ngrams, bigramas, freq_bigram  

In [3]:
url = 'https://repository.javeriana.edu.co/handle/10554/10014/recent-submissions'

In [4]:
options = webdriver.ChromeOptions()
#Le agregamos argumentos
options.add_argument('--incognito')
#Esto va a abrir un navegador vacío que dice que un software automatizado controla 
driver = webdriver.Chrome(executable_path='../chromedriver', options=options)

driver.get(url)

In [5]:
def obtener_tesis(driver):
    
    #Creamos una lista vacía para que contenga todas las tesis 
    tesis_total = []
    # Obtenemos los impares
    tesis_total.append(driver.find_elements_by_xpath('//li[@class="ds-artifact-item odd"]'))
    # Obtenemos los pares
    tesis_total.append(driver.find_elements_by_xpath('//li[@class="ds-artifact-item even"]'))
    # Transformamos la lista de listas en una sola lista 
    tesis_total =[t for l in tesis_total for t in l]
    return tesis_total

In [6]:
info = []
def obtener_info(driver):
    tesis_totales = obtener_tesis(driver)
    print('iniciando scraping...')
    for tesis in tesis_totales:
        # Obtenemos fecha
        fecha = tesis.find_element_by_xpath('.//span[@class="date"]').text
        # Obtenemos título
        titulo = tesis.find_element_by_xpath('.//h4[@class="artifact-title"]').text
        info_dict={'Fecha': fecha, 'Título': titulo}
        info.append(info_dict)
    return info

In [10]:
def main (driver):
    for x in range (1,50):
        obtener_info(driver)
        next_page(driver)
    return info

In [11]:
main(driver)

iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando 

[{'Fecha': '2020-01-28',
  'Título': 'Capacidades institucionales y comunitarias en torno al goce efectivo del derecho al agua. Caso de estudio : San Antonio del Tequendama, Cundinamarca'},
 {'Fecha': '2020-02-24',
  'Título': 'La negociación como mecanismo de solución de controversias en el sistema internacional. ¿Facultad u obligación? Estudio sobre el fallo de la Corte Internacional de Justicia en el caso de (Bolivia C. Chile)'},
 {'Fecha': '2020-04-14',
  'Título': 'Twitter como escenario de violencia simbólica : mujeres líderes de opinión pública en colombia víctimas de los estigmas y estereotipos'},
 {'Fecha': '2019-07-22',
  'Título': 'Más allá de los doscientos cuarenta caracteres : análisis de las redes discursivas en Twitter durante las elecciones presidenciales de Colombia 2018'},
 {'Fecha': '2020-03-04',
  'Título': 'La Región en Colombia : el caso de la Región Caribe'},
 {'Fecha': '2019-07-11',
  'Título': 'Democracias enfrentadas : la participación ciudadana en la planeac

In [12]:
info

[{'Fecha': '2020-01-28',
  'Título': 'Capacidades institucionales y comunitarias en torno al goce efectivo del derecho al agua. Caso de estudio : San Antonio del Tequendama, Cundinamarca'},
 {'Fecha': '2020-02-24',
  'Título': 'La negociación como mecanismo de solución de controversias en el sistema internacional. ¿Facultad u obligación? Estudio sobre el fallo de la Corte Internacional de Justicia en el caso de (Bolivia C. Chile)'},
 {'Fecha': '2020-04-14',
  'Título': 'Twitter como escenario de violencia simbólica : mujeres líderes de opinión pública en colombia víctimas de los estigmas y estereotipos'},
 {'Fecha': '2019-07-22',
  'Título': 'Más allá de los doscientos cuarenta caracteres : análisis de las redes discursivas en Twitter durante las elecciones presidenciales de Colombia 2018'},
 {'Fecha': '2020-03-04',
  'Título': 'La Región en Colombia : el caso de la Región Caribe'},
 {'Fecha': '2019-07-11',
  'Título': 'Democracias enfrentadas : la participación ciudadana en la planeac

In [13]:
driver.close()

Convertimos el archivo info en un DataFrame

In [14]:
df = pd.DataFrame(info)

Cambiamos los valores dentro de fecha a años 

In [21]:
regex = r'(\d{1,4})'
df['Fecha']= df['Fecha'].str.extract(regex).squeeze().str.zfill(4)

In [22]:
df.head()

,Fecha,Título
0,2020,Capacidades institucionales y comunitarias en ...
1,2020,La negociación como mecanismo de solución de c...
2,2020,Twitter como escenario de violencia simbólica ...
3,2019,Más allá de los doscientos cuarenta caracteres...
4,2020,La Región en Colombia : el caso de la Región C...


Volvemos el valor en fecha un integrer

In [23]:
df['Fecha'] = df['Fecha'].astype(int)

In [24]:
df.head()

,Fecha,Título
0,2020,Capacidades institucionales y comunitarias en ...
1,2020,La negociación como mecanismo de solución de c...
2,2020,Twitter como escenario de violencia simbólica ...
3,2019,Más allá de los doscientos cuarenta caracteres...
4,2020,La Región en Colombia : el caso de la Región C...


Limpiamos los títulos de las tesis de signos de puntuación y signos speciales. Además se tokenizan los títulos y se ponen todas las palabras en minúscula

In [41]:
def clean_titulo(df):
    """Esta función recibe un Data Frame y a la columna con """
    # Definimos los signos de puntiación en una variable
    punt_espa = string.punctuation + "–¡¿”“•\r"
    
    # Definimos función para quitar los signos de puntiación 
    def sin_punt(x):
        x = ''.join([i for i in x if i not in (punt_espa)])
        return x
    
    # Aplicamos la función para quitar los signos
    df['Título cleaned'] = df['Título'].apply(sin_punt)
    
    # Definimos las stopwords
    stopwords_esp = stopwords.words('spanish')
    stopwords_esp = stopwords_esp + ['estudio', 'caso', 'análisis', 'bogotá', 'periodo', 'incidencia', 'colombia', 'político', 'hacia', 'impacto', 'the', 'and', 'aproximación', 'influencia', 'factores', 'medio', 'interés', 'programa', 'influencia', 'partir', 'colombiano', 'relación', 'marco', 'concepto', 'gobierno', 'periodo', 'departamento', 'ciudad','What’s', 'love', 'got', 'to', 'do', 'with', 'it', 'of', 'in', 'enero', 'febrero', 'marzo', 'abril', 'mayo', 'junio', 'julio', 'agosto', 'septiembre', 'octubre', 'noviembre', 'diciembre', 'ddr']
    
    
    # Ponemos las palabras todas en minúscula y las separamos
    df['Título cleaned'] = df['Título cleaned'].str.lower().str.split()
    
    # Eliminamos las stopwords
    df['Título cleaned'] = df['Título cleaned'].apply(lambda x: [item for item in x if item not in stopwords_esp])
    
    
    return df

In [42]:
clean_titulo(df)

,Fecha,Título,Título cleaned,bigrams
0,2020,Capacidades institucionales y comunitarias en ...,"[capacidades, institucionales, comunitarias, t...","[(capacidades, institucionales), (instituciona..."
1,2020,La negociación como mecanismo de solución de c...,"[negociación, mecanismo, solución, controversi...","[(negociación, mecanismo), (mecanismo, solució..."
2,2020,Twitter como escenario de violencia simbólica ...,"[twitter, escenario, violencia, simbólica, muj...","[(twitter, escenario), (escenario, violencia),..."
3,2019,Más allá de los doscientos cuarenta caracteres...,"[allá, doscientos, cuarenta, caracteres, redes...","[(allá, doscientos), (doscientos, cuarenta), (..."
4,2020,La Región en Colombia : el caso de la Región C...,"[región, región, caribe]","[(región, colombia), (colombia, región), (regi..."
...,...,...,...,...
959,2009,"Abriendo la ""Caja negra"" de la guerra civil : ...","[abriendo, caja, negra, guerra, civil, mecanis...","[(abriendo, caja), (caja, negra), (negra, guer..."
960,2008,Determinantes políticos del desempeño fiscal. ...,"[determinantes, políticos, desempeño, fiscal, ...","[(determinantes, políticos), (políticos, desem..."
961,2009,El país que nos dejaron : cómo desestabilizaro...,"[país, dejaron, cómo, desestabilizaron, cartel...","[(país, dejaron), (dejaron, cómo), (cómo, dese..."
962,2005,Revoluciones democráticas : la revolución nara...,"[revoluciones, democráticas, revolución, naran...","[(revoluciones, democráticas), (democráticas, ..."


Extraemos los bigramas de los títulos

In [43]:
def bigramas(df):
    """Esta función encuentra los bigramas por row dentro de una colomna seleccionada de un dataframe"""
    df['bigrams'] = df['Título cleaned'].map(lambda x: find_ngrams(x, 2))
    return df

In [44]:
bigramas(df)

,Fecha,Título,Título cleaned,bigrams
0,2020,Capacidades institucionales y comunitarias en ...,"[capacidades, institucionales, comunitarias, t...","[(capacidades, institucionales), (instituciona..."
1,2020,La negociación como mecanismo de solución de c...,"[negociación, mecanismo, solución, controversi...","[(negociación, mecanismo), (mecanismo, solució..."
2,2020,Twitter como escenario de violencia simbólica ...,"[twitter, escenario, violencia, simbólica, muj...","[(twitter, escenario), (escenario, violencia),..."
3,2019,Más allá de los doscientos cuarenta caracteres...,"[allá, doscientos, cuarenta, caracteres, redes...","[(allá, doscientos), (doscientos, cuarenta), (..."
4,2020,La Región en Colombia : el caso de la Región C...,"[región, región, caribe]","[(región, región), (región, caribe)]"
...,...,...,...,...
959,2009,"Abriendo la ""Caja negra"" de la guerra civil : ...","[abriendo, caja, negra, guerra, civil, mecanis...","[(abriendo, caja), (caja, negra), (negra, guer..."
960,2008,Determinantes políticos del desempeño fiscal. ...,"[determinantes, políticos, desempeño, fiscal, ...","[(determinantes, políticos), (políticos, desem..."
961,2009,El país que nos dejaron : cómo desestabilizaro...,"[país, dejaron, cómo, desestabilizaron, cartel...","[(país, dejaron), (dejaron, cómo), (cómo, dese..."
962,2005,Revoluciones democráticas : la revolución nara...,"[revoluciones, democráticas, revolución, naran...","[(revoluciones, democráticas), (democráticas, ..."


Guardamos el data frame en un archivo pickle para después trabajar con él

In [45]:
df.to_pickle('repo_javeriana.pkl')